In [108]:
## Package
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [109]:
from typing import List
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

In [25]:
import os

In [192]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

In [114]:
import re
vector = {}
pattern = re.compile(r"(\w+).+")
with open("vectors.txt",'r',encoding="utf-8") as file:
    line = file.readline()
    while line:
        m=re.match(pattern,line)
        #print(line[len(m.group(1)):].split())
        if m:
            vector[m.group(1)]=line[len(m.group(1)):].split()
        line = file.readline()


In [115]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, y_test = get_X_Y_from_root(root_test)

In [116]:
X_train_sample = X_train[:500]
y_train_sample = y_train[:500]

In [194]:
v = CountVectorizer()
D = X_train_sample
testy = v.fit_transform(D)

In [201]:
clf = LinearSVC()
clf.fit(testy,y_train_sample)

LinearSVC()

In [197]:
testy.shape

(500, 12624)

In [199]:
testy.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [119]:
# Global parameters
#root folder
root_folder='.'
glove_filename='vectors.txt'

# Variable for data directory
glove_path = os.path.abspath(glove_filename)


In [120]:
# We just need to run this code once, the function glove2word2vec saves the Glove embeddings in the word2vec format 
# that will be loaded in the next section
from gensim.scripts.glove2word2vec import glove2word2vec
glove_filename='vectors.txt'
#glove_input_file = glove_filename
word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec(glove_path, word2vec_output_file)

/tmp/ipykernel_188/3652708010.py:7: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_path, word2vec_output_file)


(41153, 50)

In [143]:
from gensim.models import KeyedVectors
word2vec_output_file = glove_filename+'.word2vec'
glove = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [144]:
#Show a word embedding
print('Madame: ',glove.get_vector('madame'))

Madame:  [ 0.013859  0.451532 -0.487101  0.168849 -0.327894  0.146996 -0.224299
 -0.178038 -0.070914 -0.51272  -0.421458 -0.469794 -0.545544  0.139949
 -0.006503 -0.140751  0.333008 -0.048612  0.200209  0.376872  0.138722
 -0.271306  0.140596  0.33297   0.664384  0.201292 -0.005523 -0.746854
 -0.041821 -0.062128 -0.132737 -0.275511  0.495469  0.177609 -0.082183
 -0.180198  0.092252  0.044848  0.353179  0.449488 -0.322928 -0.258066
 -0.727073  0.10923   0.008607  0.232664  0.044286  0.068772  0.311111
  0.15182 ]


In [145]:
for index, word in enumerate(glove.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(glove.index_to_key)} is {word}")

word #0/41153 is de
word #1/41153 is la
word #2/41153 is et
word #3/41153 is le
word #4/41153 is à
word #5/41153 is les
word #6/41153 is des
word #7/41153 is que
word #8/41153 is en
word #9/41153 is du


In [173]:
words = set(model.index_to_key)

In [181]:
model[0]

array([ 0.756286,  0.357416,  0.110608,  0.386516,  1.550777,  1.69021 ,
       -0.042318, -1.408475,  1.903917,  0.546989,  0.639621,  0.19015 ,
        0.004072,  0.352885,  0.186876,  0.301915, -1.090066, -0.191985,
        0.141774,  0.872651,  0.310097,  1.655799, -0.424185,  0.929607,
       -0.435726,  0.742252,  0.304116,  1.093469, -0.420415, -0.042222,
        1.265649,  2.294702, -0.523597, -0.644814,  0.603675, -0.493675,
       -0.776827,  0.226664,  0.709328, -1.273558,  0.6168  ,  0.541127,
        0.451895, -0.272934, -1.010776,  0.852358,  0.003264,  0.216668,
       -1.258991,  0.387364], dtype=float32)

In [187]:
X_train_vect = np.array([np.array([model[i] for i in ls if i in words])
                         for ls in X_train])

In [188]:
def GloveVectorizer(model,X_train):
    return np.array([np.array([model[i] for i in ls if i in words])
                         for ls in X_train])

In [189]:
X_vect = GloveVectorizer(model,X_train_sample)

/tmp/ipykernel_188/1248266255.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([np.array([model[i] for i in ls if i in words])


In [186]:
X_vect.shape

(500,)

In [193]:
from sklearn.ensemble import RandomForestClassifier
clf = LinearSVC()
Xtest = clf.fit(X_vect,y_train_sample)

ValueError: setting an array element with a sequence.

In [171]:
data = ["La Présidente parle."]
n=0
m = glove.get_vector('président')
X = np.zeros((len(data),m.shape[0]))

for phrase in data:
    tokens = phrase.split()
    vecs = []
    for token in tokens:
        v = glove.get_vector(token)
        print(token,v)
        vecs.append(v)
    vecs = np.array(vecs)
    X[n] = vecs.mean(axis=0)
    n=n+1


La [-0.356746 -0.763595 -0.439038  0.400101  0.348126  0.664703  0.691902
 -1.370866  1.874059  0.008522  1.051088 -0.168902 -0.985847  0.559935
 -0.119273  0.312639 -1.364021 -0.140409 -0.977615  1.41729   0.493159
  1.043272  0.192097 -0.421588  0.778566  0.823617 -0.90677   1.003853
  0.504122 -0.020136  0.480741  1.802187 -0.216162 -0.827201 -0.15117
  0.573967 -0.299327 -0.202961  0.406103 -0.561666  0.847018  0.813715
 -0.520787  0.008835 -0.471773  0.828857 -0.236493  0.549296 -1.436775
 -0.271421]
Présidente [-2.985920e-01 -5.042320e-01 -2.069940e-01 -3.398690e-01  4.104580e-01
  2.722290e-01  4.177560e-01 -5.758040e-01  2.407050e-01  4.824950e-01
 -3.483500e-02  9.440000e-04 -1.034850e+00 -9.299000e-03 -8.051290e-01
 -6.766020e-01 -1.368750e-01 -7.404110e-01  8.883000e-02  3.501070e-01
  1.051480e-01  1.635700e-01  1.031557e+00  1.145090e-01 -2.906010e-01
 -2.816790e-01 -5.765150e-01 -5.500540e-01 -4.253330e-01 -3.232340e-01
 -3.044530e-01  1.001771e+00  1.047254e+00  2.982190

In [133]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('président')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [137]:
# Set a word vectorizer
vectorizer = Word2VecVectorizer(model)

Loading in word vectors...
Finished loading in word vectors


In [138]:
# Get the sentence embeddings for the train dataset
X_train = vectorizer.fit_transform(X_train_sample)
y_train = y_train_sample
# Get the sentence embeddings for the test dataset
#X_test = vectorizer.transform(X_test)
#Y_test = test_labels

Numer of samples with no words found: 0 / 500


In [31]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('président')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [32]:
X_train[0]

"Monsieur le Président, j'ai toujours fait preuve d'un certain scepticisme quant aux comportements moralistes et policiers qui encouragent carrément la délation et qui permettent d'éviter des cas de fraude seulement par le biais de la création d'instruments de contrôle toujours plus envahissants par rapport aux personnes. Nous pouvons certainement - mon groupe et moi-même - soutenir avec conviction le rapport du président Napolitano. La modification du règlement et la proposition de décision répondent en effet sérieusement à une préoccupation qui a été soulevée récemment et que M. Dell'Alba a été le dernier à exprimer : ne pas soumettre les députés à des contrôles excessifs et arbitraires. Je ne vois aucune raison pour retarder davantage cette décision. Si un tel abus devait se produire, le Parlement européen et les députés disposent de larges possibilités pour se défendre. Des procédures internes plus transparentes et plus claires rendront évidemment les choses plus faciles. Nous devo

In [33]:
# Set a word vectorizer
vectorizer = Word2VecVectorizer(model)
# Get the sentence embeddings for the train dataset
X_train = vectorizer.fit_transform(X_train_sample)
y_train = y_train_sample
# Get the sentence embeddings for the test dataset
X_test = vectorizer.transform(X_test)
#Y_test = test_labels

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 0 / 500
Numer of samples with no words found: 2 / 12917


In [34]:
print(X_train.shape)

(500, 50)


In [35]:
from sklearn.ensemble import RandomForestClassifier

# create the model, train it, print scores
clf = RandomForestClassifier(n_estimators=200)

clf.fit(X_train, y_train)

print("train score:", clf.score(X_train, y_train))



train score: 1.0


In [37]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [44]:
root_folder='.'
# Global parameters
#root folder
root_folder='.'
glove_filename='vectors.txt'

# Variable for data directory
glove_path = os.path.abspath(glove_filename)


In [46]:
print(glove_path)

/mnt/c/Users/Julie/OneDrive/Bureau/Apprentissage_Artificiel/vectors.txt


In [58]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file =glove_path
tmp_file = "test_word2vec.txt"
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)
model_bis = KeyedVectors.load_word2vec_format(tmp_file, binary=False)

/tmp/ipykernel_188/2457025952.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, tmp_file)


In [74]:
model_bis

In [75]:
result = model_bis.most_similar(positive=['Présidente', 'Président'], negative=['femme'], topn=1)

print('Most similar word to Roi + Reine: ', result)

Most similar word to Roi + Reine:  [('exercice,', 0.8002204895019531)]


In [94]:
class GloveVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vectorizer, ngram_range:tuple=(1,1)) -> None:
        super().__init__()
        self.vectorizer = vectorizer
        self.ngram_range = ngram_range
    def fit(self, X, y=None):
        print(f">>> Vectorizer.fit() called with vectorizer={self.vectorizer} and ngram_range={self.ngram_range}.")
        self.vectorizer.fit(X)
        return self 
    def transform(self, X, y=None):
        print(f">>> Vectorizer.transform() called with vectorizer={self.vectorizer} and ngram_range={self.ngram_range}.")
        X_ = X.copy()
        X_vect_ = self.vectorizer.transform(X_)
        X_vect_ = X_vect_.toarray()
        return X_vect_

In [99]:
class GloveVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('président')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [100]:
# Set a word vectorizer
vectorizer = GloveVectorizer(model_bis)
# Get the sentence embeddings for the train dataset
#X_train = vectorizer.fit_transform(X_train_sample)
#y_train = y_train_sample
# Get the sentence embeddings for the test dataset
#X_test = vectorizer.transform(X_test)
#Y_test = test_labels

Loading in word vectors...
Finished loading in word vectors


In [101]:
vectorizer

In [106]:
X_train = vectorizer.fit_transform(X_train_sample)

Numer of samples with no words found: 0 / 500


In [107]:
print(X_train)

[[ 0.01528353 -0.17387539 -0.04330047 ...  0.27416426 -0.94076514
  -0.25462666]
 [-0.03737077 -0.02463203  0.14741205 ...  0.22106427 -0.7972185
  -0.31570014]
 [ 0.0171264  -0.18690102 -0.07965815 ...  0.26617157 -0.91300488
  -0.29518124]
 ...
 [-0.05366648 -0.19655336 -0.03869827 ...  0.31806138 -0.90246582
  -0.30373943]
 [ 0.11446893 -0.12268434  0.00983311 ...  0.34728044 -0.79701775
  -0.09281527]
 [-0.01101418 -0.15832914  0.01620665 ...  0.18718338 -0.85960853
  -0.27101207]]
